In [1]:
import torch.autograd
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision import datasets
from torchvision.utils import save_image
import os
import wandb

In [2]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="gan-mnist",

    # track hyperparameters and run metadata
    config={
        "learning_rate": 0.0003,
        "epochs": 100,
    })

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jianpingzhou0927 (sjtuzjp). Use `wandb login --relogin` to force relogin


In [3]:
# 设备配置
torch.cuda.set_device(1) # 这句用来设置pytorch在哪块GPU上运行
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# 创建文件夹
if not os.path.exists('./img'):
    os.mkdir('./img')

cuda


In [4]:
def to_img(x):
    out = 0.5 * (x + 1)
    out = out.clamp(0, 1)  # Clamp函数可以将随机变化的数值限制在一个给定的区间[min, max]内：
    out = out.view(-1, 1, 28, 28)  # view()函数作用是将一个多行的Tensor,拼接成一行
    return out

In [5]:
# 超参数设置
batch_size = 128
num_epoch = 100
z_dimension = 100
# 图像预处理
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))  # (x-mean) / std
])

In [6]:
# mnist dataset train数据集下载
mnist = datasets.MNIST(root='./data/mnist',
                       train=True,
                       transform=img_transform,
                       download=True)

# data loader 数据载入
dataloader = torch.utils.data.DataLoader(dataset=mnist,
                                         batch_size=batch_size,
                                         shuffle=True)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw


  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw


  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw
Processing...


/home/zhoujianping/miniconda3/envs/torch3090/lib/python3.8/site-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [7]:
# 定义判别器  #####Discriminator######使用多层网络来作为判别器
# 将图片28x28展开成784，然后通过多层感知器，中间经过斜率设置为0.2的LeakyReLU激活函数，
# 最后接sigmoid激活函数得到一个0到1之间的概率进行二分类。
class discriminator(nn.Module):

    def __init__(self):
        super(discriminator, self).__init__()
        self.dis = nn.Sequential(
            nn.Linear(784, 256),  # 输入特征数为784，输出为256
            nn.LeakyReLU(0.2),  # 进行非线性映射
            nn.Linear(256, 256),  # 进行一个线性映射
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()  # 也是一个激活函数，二分类问题中，
            # sigmoid可以班实数映射到【0,1】，作为概率值，
            # 多分类用softmax函数
        )

    def forward(self, x):
        x = self.dis(x)
        return x


In [8]:
# ###### 定义生成器 Generator #####
# 输入一个100维的0～1之间的高斯分布，然后通过第一层线性变换将其映射到256维,
# 然后通过LeakyReLU激活函数，接着进行一个线性变换，再经过一个LeakyReLU激活函数，
# 然后经过线性变换将其变成784维，最后经过Tanh激活函数是希望生成的假的图片数据分布能够在-1～1之间。
class generator(nn.Module):

    def __init__(self):
        super(generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(100, 256),  # 用线性变换将输入映射到256维
            nn.ReLU(True),  # relu激活
            nn.Linear(256, 256),  # 线性变换
            nn.ReLU(True),  # relu激活
            nn.Linear(256, 784),  # 线性变换
            nn.Tanh()  # Tanh激活使得生成数据分布在【-1,1】之间，因为输入的真实数据的经过transforms之后也是这个分布
        )

    def forward(self, x):
        x = self.gen(x)
        return x


In [9]:
# 创建对象
D = discriminator().to(device)
G = generator().to(device)
# if torch.cuda.is_available():
#     D = D.cuda()
#     G = G.cuda()

逐步打印训练过程的vector shape

In [10]:
# 首先需要定义loss的度量方式  （二分类的交叉熵）
# 其次定义优化函数,优化函数的学习率为0.0003
criterion = nn.BCELoss()  # 是单目标二分类交叉熵函数
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0003)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0003)

for i, (img, _) in enumerate(dataloader):
    num_img = img.size(0)
    print(num_img)  # 128
    # view()函数作用是将一个多行的Tensor,拼接成一行
    # 第一个参数是要拼接的tensor,第二个参数是-1
    # =============================训练判别器==================
    img = img.view(num_img, -1)  # 将图片展开为28*28=784
    print(img.shape)    # (128,784)
    real_img = Variable(img).cuda()  # 将tensor变成Variable放入计算图中
    real_label = Variable(torch.ones(num_img)).cuda()  # 定义真实的图片label为1
    fake_label = Variable(torch.zeros(num_img)).cuda()  # 定义假的图片的label为0

    # ########判别器训练train#####################
    # 分为两部分：1、真的图像判别为真；2、假的图像判别为假

    # 计算真实图片的损失
    real_out = D(real_img)  # 将真实图片放入判别器中
    print(real_out.shape)
    real_out = real_out.squeeze()  # (128,1) -> (128,)
    print(real_out.shape)   # 128

    d_loss_real = criterion(real_out, real_label)  # 得到真实图片的loss
    print(d_loss_real)
    real_scores = real_out  # 得到真实图片的判别值，输出的值越接近1越好
    print(real_scores.shape)    # 128

    # 计算假的图片的损失
    z = Variable(torch.randn(num_img, z_dimension)).cuda()  # 随机生成一些噪声
    fake_img = G(
        z).detach()  # 随机噪声放入生成网络中，生成一张假的图片。 # 避免梯度传到G，因为G不用更新, detach分离
    fake_out = D(fake_img)  # 判别器判断假的图片
    fake_out = fake_out.squeeze()  # (128,1) -> (128,)
    d_loss_fake = criterion(fake_out, fake_label)  # 得到假的图片的loss
    fake_scores = fake_out  # 得到假图片的判别值，对于判别器来说，假图片的损失越接近0越好

    # 损失函数和优化
    d_loss = d_loss_real + d_loss_fake  # 损失包括判真损失和判假损失
    d_optimizer.zero_grad()  # 在反向传播之前，先将梯度归0
    d_loss.backward()  # 将误差反向传播
    d_optimizer.step()  # 更新参数


    # ==================训练生成器============================
    # ###############################生成网络的训练###############################
    # 原理：目的是希望生成的假的图片被判别器判断为真的图片，
    # 在此过程中，将判别器固定，将假的图片传入判别器的结果与真实的label对应，
    # 反向传播更新的参数是生成网络里面的参数，
    # 这样可以通过更新生成网络里面的参数，来训练网络，使得生成的图片让判别器以为是真的
    # 这样就达到了对抗的目的
    # 计算假的图片的损失
    z = Variable(torch.randn(num_img, z_dimension)).cuda()  # 得到随机噪声
    print(z.shape)  # (128,100)
    fake_img = G(z)  # 随机噪声输入到生成器中，得到一副假的图片
    print(fake_img.shape)   # (128,784)
    output = D(fake_img)  # 经过判别器得到的结果
    print(output.shape) # (128,1)
    output = output.squeeze()
    g_loss = criterion(output, real_label)  # 得到的假的图片与真实的图片的label的loss
    # bp and optimize
    g_optimizer.zero_grad()  # 梯度归0
    g_loss.backward()  # 进行反向传播
    g_optimizer.step()  # .step()一般用在反向传播后面,用于更新生成网络的参数
    break


128
torch.Size([128, 784])
torch.Size([128, 1])
torch.Size([128])
tensor(0.6788, device='cuda:1', grad_fn=<BinaryCrossEntropyBackward>)
torch.Size([128])
torch.Size([128, 100])
torch.Size([128, 784])
torch.Size([128, 1])


In [11]:
# 首先需要定义loss的度量方式  （二分类的交叉熵）
# 其次定义 优化函数,优化函数的学习率为0.0003
criterion = nn.BCELoss()  # 是单目标二分类交叉熵函数
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0003)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0003)

# ##########################进入训练##判别器的判断过程#####################
for epoch in range(num_epoch):  # 进行多个epoch的训练
    for i, (img, _) in enumerate(dataloader):
        num_img = img.size(0)
        # view()函数作用是将一个多行的Tensor,拼接成一行
        # 第一个参数是要拼接的tensor,第二个参数是-1
        # =============================训练判别器==================
        img = img.view(num_img, -1)  # 将图片展开为28*28=784
        real_img = Variable(img).cuda()  # 将tensor变成Variable放入计算图中
        real_label = Variable(torch.ones(num_img)).cuda()  # 定义真实的图片label为1
        fake_label = Variable(torch.zeros(num_img)).cuda()  # 定义假的图片的label为0

        # ########判别器训练train#####################
        # 分为两部分：1、真的图像判别为真；2、假的图像判别为假
        # 计算真实图片的损失
        real_out = D(real_img)  # 将真实图片放入判别器中
        real_out = real_out.squeeze()  # (128,1) -> (128,)
        d_loss_real = criterion(real_out, real_label)  # 得到真实图片的loss
        real_scores = real_out  # 得到真实图片的判别值，输出的值越接近1越好
        # 计算假的图片的损失
        z = Variable(torch.randn(num_img, z_dimension)).cuda()  # 随机生成一些噪声
        fake_img = G(
            z).detach()  # 随机噪声放入生成网络中，生成一张假的图片。 # 避免梯度传到G，因为G不用更新, detach分离
        fake_out = D(fake_img)  # 判别器判断假的图片，
        fake_out = fake_out.squeeze()  # (128,1) -> (128,)
        d_loss_fake = criterion(fake_out, fake_label)  # 得到假的图片的loss
        fake_scores = fake_out  # 得到假图片的判别值，对于判别器来说，假图片的损失越接近0越好
        # 损失函数和优化
        d_loss = d_loss_real + d_loss_fake  # 损失包括判真损失和判假损失
        d_optimizer.zero_grad()  # 在反向传播之前，先将梯度归0
        d_loss.backward()  # 将误差反向传播
        d_optimizer.step()  # 更新参数

        # ==================训练生成器============================
        # ###############################生成网络的训练###############################
        # 原理：目的是希望生成的假的图片被判别器判断为真的图片，
        # 在此过程中，将判别器固定，将假的图片传入判别器的结果与真实的label对应，
        # 反向传播更新的参数是生成网络里面的参数，
        # 这样可以通过更新生成网络里面的参数，来训练网络，使得生成的图片让判别器以为是真的
        # 这样就达到了对抗的目的
        # 计算假的图片的损失
        z = Variable(torch.randn(num_img, z_dimension)).cuda()  # 得到随机噪声
        fake_img = G(z)  # 随机噪声输入到生成器中，得到一副假的图片
        output = D(fake_img)  # 经过判别器得到的结果
        output = output.squeeze()
        g_loss = criterion(output, real_label)  # 得到的假的图片与真实的图片的label的loss
        # bp and optimize
        g_optimizer.zero_grad()  # 梯度归0
        g_loss.backward()  # 进行反向传播
        g_optimizer.step()  # .step()一般用在反向传播后面,用于更新生成网络的参数

        # 打印中间的损失
        if (i + 1) % 100 == 0:
            print('Epoch[{}/{}],d_loss:{:.6f},g_loss:{:.6f} '
                  'D real: {:.6f},D fake: {:.6f}'.format(
                      epoch,
                      num_epoch,
                      d_loss.data.item(),
                      g_loss.data.item(),
                      real_scores.data.mean(),
                      fake_scores.data.mean()  # 打印的是真实图片的损失均值
                  ))
        if epoch == 0:
            real_images = to_img(real_img.cpu().data)
            save_image(real_images, './img/real_images.png')
    wandb.log({"generator_loss": g_loss, "discriminator_loss": d_loss})
    fake_images = to_img(fake_img.cuda().data)
    save_image(fake_images, './img/fake_images-{}.png'.format(epoch + 1))
wandb.finish()

Epoch[0/100],d_loss:0.188616,g_loss:3.179043 D real: 0.956892,D fake: 0.130629
Epoch[0/100],d_loss:0.024438,g_loss:4.703603 D real: 0.997647,D fake: 0.021758
Epoch[0/100],d_loss:0.210749,g_loss:5.839513 D real: 0.970132,D fake: 0.132535
Epoch[0/100],d_loss:0.088411,g_loss:5.436432 D real: 0.980465,D fake: 0.061762
Epoch[1/100],d_loss:0.031820,g_loss:5.491758 D real: 0.981815,D fake: 0.010042
Epoch[1/100],d_loss:0.017488,g_loss:6.660955 D real: 0.998567,D fake: 0.015685
Epoch[1/100],d_loss:0.348130,g_loss:4.930775 D real: 0.887380,D fake: 0.111554
Epoch[1/100],d_loss:0.155825,g_loss:5.267170 D real: 0.939481,D fake: 0.044494
Epoch[2/100],d_loss:1.146036,g_loss:4.903194 D real: 0.868736,D fake: 0.457917
Epoch[2/100],d_loss:0.090659,g_loss:5.787194 D real: 0.970833,D fake: 0.044559
Epoch[2/100],d_loss:0.308463,g_loss:4.019037 D real: 0.913866,D fake: 0.132509
Epoch[2/100],d_loss:0.198864,g_loss:3.539353 D real: 0.951901,D fake: 0.114046
Epoch[3/100],d_loss:0.850701,g_loss:2.237185 D real:

discriminator_loss,▁▄▄█▄▄▂▃▄▄▃▂▂▂▂▃▅▃▃▄▃▃▄▄▃▅▄▄▃▄▅▄▄▅▄▃▅▄▅▃
generator_loss,█▆▂▅▁▅▅▆▅▆▄▇▆▅▅▅▅▅▄▆▄▄▄▃▅▂▃▃▂▃▃▂▃▂▂▂▁▃▁▂
discriminator_loss,0.59048
generator_loss,1.94841


In [12]:
# 保存模型
torch.save(G.state_dict(), './generator.pth')
torch.save(D.state_dict(), './discriminator.pth')